<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/API_Load_Testing_using_Cloudwatch_Custom_Metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install boto3 requests pandas

In [ ]:
import time
import json
import pandas as pd
import requests
import boto3

In [ ]:

# Initialize CloudWatch client
cloudwatch = boto3.client('cloudwatch', aws_access_key_id='', aws_secret_access_key='', region_name='us-east-1')

# Read the CSV file
csv_file = "/content/extarcted_data_for_testing30k.csv"  # Replace with the path to your CSV file
data = pd.read_csv(csv_file)
data = data.head(300)

# API URL and headers
api_url = "https://hccreference.com/api/search"
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8,hi;q=0.7',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'origin': 'https://hccreference.com',
    'pragma': 'no-cache',
    'referer': 'https://hccreference.com/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
}

def send_metrics_to_cloudwatch(metric_name, value, unit='Milliseconds', namespace='API_RESPONSE_DATA_CUSTOM_METRIC'):
    try:
        cloudwatch.put_metric_data(
            MetricData=[
                {
                    'MetricName': metric_name,
                    'Unit': unit,
                    'Value': value
                },
            ],
            Namespace=namespace
        )
        print(f"Metric {metric_name} sent successfully.")
    except Exception as e:
        print(f"Error sending metric {metric_name}: {e}")

def make_api_requests(data):
    for index, row in data.iterrows():
        payload = {
            "dx_hcc": row['dx_hcc'],
            "dos_year": row['dos_year'],
            "drf": row['drf'],
            "search": row['search']
        }
        try:
            start_time = time.perf_counter()
            response = requests.post(api_url, headers=headers, json=payload)
            end_time = time.perf_counter()

            response_time = (end_time - start_time) * 1000  # Convert to milliseconds
            print(f"Row {index + 1}: Response time {response_time:.2f} ms")

            send_metrics_to_cloudwatch('API_Response_Time', response_time)

            if response.status_code != 200:
                print(f"Row {index + 1}: API error - {response.status_code}")
                send_metrics_to_cloudwatch('API_Errors', 1, unit='Count')
        except Exception as e:
            print(f"Error processing row {index + 1}: {e}")
            send_metrics_to_cloudwatch('API_Errors', 1, unit='Count')

# Run the API requests
make_api_requests(data)
